# Raster Analysis with Rioxarray

## Introduction

## Learning Objectives

## Installing Rioxarray

## Loading Georeferenced Raster Data

### Inspecting the Dataset

### Checking CRS and Transform Information

## Basic Geospatial Operations

### Reprojecting a Dataset

### Clipping a Raster

## Working with Spatial Dimensions

### Resampling

### Extracting Spatial Subsets

## Visualization of Georeferenced Data

## Saving Data

## Handling NoData Values

## Reprojecting to Multiple CRS

## Basic Band Math

## Key Takeaways

## Exercises

### Sample Dataset

### Exercise 1: Load and Inspect a Raster Dataset

### Exercise 2: Reproject the Raster to a New CRS

### Exercise 3: Clip the Raster Using a Bounding Box

### Exercise 4: Mask the Raster Using a Vector Dataset

### Exercise 5: Resample the Raster to a Different Resolution